Install tensorflow gpu

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 53kB/s 


In [ ]:
!nvidia-smi

Sat Oct 24 04:52:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
tf.__version__

'2.3.1'

Kết nối tới google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Trước khi thực hiện train cần chạy một số modules này trước phục vục cho việc code

In [ ]:
import os
import cv2
import sys
import zipfile

def fun_print(name: str, value) -> None:
    print('@ Deep Learning> ', name)
    print(value)


def fun_getFileNames(path: str) -> list:
    return os.listdir(path)


def fun_showVideoPath(path: str, delay: int = 25) -> None:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    while True:
        if not isContinue:
            break
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break
        isContinue, frame = cap.read()

    cap.release()
    cv2.destroyAllWindows()


def fun_getFramesOfVideo(path: str, count: int = 20) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while count > 0:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
        count -= 1
    cap.release()
    cv2.destroyAllWindows()
    return imgs

def fun_getFramesOfVideo_ALL(path: str) -> list:
    cap = cv2.VideoCapture(path)
    isContinue, frame = cap.read()
    imgs = []
    while True:
        if not isContinue:
            break
        imgs.append(frame)
        isContinue, frame = cap.read()
    cap.release()
    cv2.destroyAllWindows()
    return imgs


def fun_showVideoFrames(frames: list, delay: int = 25) -> None:
    for frame in frames:
        cv2.imshow('frame', frame)
        if cv2.waitKey(delay=delay) & 0xFF == ord('q'):
            break


def fun_showVideo(source, delay: int = 25) -> None:
    if isinstance(source, str):
        fun_showVideoPath(path=source, delay=delay)
    else:
        fun_showVideoFrames(frames=source, delay=delay)


def fun_resizeFrames(frames: list, size: tuple = (224, 224)) -> list:
    imgs = []
    for frame in frames:
        try:
          fr = cv2.resize(frame, dsize=size)
        except:
          print('\r!Error To Resize of {0}'.format(len(frames)))
        imgs.append(fr)
    cv2.destroyAllWindows()
    return imgs


def fun_saveFramesToVideo(frames: list, path: str, fps: int = 25) -> bool:
    try:
        height, width, layer = frames[0].shape
        wr = cv2.VideoWriter(path, cv2.VideoWriter_fourcc(*'MJPG'), fps, (width, height))
        for frame in frames:
            wr.write(frame)
        wr.release()
        cv2.destroyAllWindows()
        return True

    except:
        fun_print(name='Write Video: '+path, value='ERROR TO WRITE VIDEO')
        return False

def fun_getSizeOfFrame(frame) -> tuple:
    height, width, layer = frame.shape
    return (width, height)

# version 1
def fun_outListVideoWithNumFrame(pathVideoLoad: str, dirToSave: str, preFixName: str, videoNameIndex: int= None, countFrame: int = 40, fps: int = 25, isShowCalculating: bool = False) -> int:
    if videoNameIndex is None:
        fun_print('fun_outListVideoWithNumFrame', 'Please input para: videoNameIndex')
        return 0

    all = 0
    countWriten = 0
    if isShowCalculating:
        fun_print('Calculator Video Out Frame', 'calculating...')
        all = fun_getFramesOfVideo_ALL(pathVideoLoad)
        all = len(all) // countFrame

    cap = cv2.VideoCapture(pathVideoLoad)
    isContinue, frame = cap.read()
    count = videoNameIndex
    while True:
        if not isContinue:
            break
        nameFile = dirToSave + preFixName + '_out_'+str(count)+'.avi'
        cFrame = countFrame
        frames = []

        # get list frame
        while cFrame > 0:
            frames.append(frame)
            isContinue, frame = cap.read()
            if not isContinue:
                break
            cFrame -= 1

        # check video enough frameCount frame ?
        if len(frames) != countFrame:
            break

        # write list frame
        res = fun_saveFramesToVideo(frames=frames, path=nameFile, fps=fps)
        countWriten += 1
        if res:
            if isShowCalculating:
                percent = countWriten / all
                mess = '\r - Writen: {0} -> Complete: {1:.1%}'.format(nameFile, percent)
                sys.stdout.write(mess)
                sys.stdout.flush()
            else:
                mess = '\r - Writen: {0} -> Complete'.format(nameFile)
                sys.stdout.write(mess)
                sys.stdout.flush()

        # done
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    return count

def fun_extractZipFile(pathFileZip: str, pathToSave: str) -> None:
    if not os.path.exists(pathToSave):
        os.makedirs(pathToSave)
    fun_print(name='Extract All ' + pathFileZip, value='Extracting...')
    if (pathFileZip.endswith('.zip')):
        zipfile.ZipFile(file=pathFileZip, mode='r').extractall(path=pathToSave)
        print('Extract Done')
    else:
        print('Please Input zip file')

def fun_print_process(count: int, max: int, mess: str = 'Processing: ') -> None:
  process = count / max
  mess = '\r - ' +  mess + ' [{0:.1%}]'.format(process)
  sys.stdout.write(mess)
  sys.stdout.flush()

Mình cần định nghĩa một số Const để dể dàng thực hiện khi mỗi lần Retrain. TẠI ĐÂY

In [ ]:
DIR_ROOT = '/content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/'
DIR_INPUT_TRAIN = DIR_ROOT + 'InputTrainTest/'
DIR_MODEL_LSTM = DIR_ROOT + 'Models/LSTM_Train_Viet2.h5'
SIZE = (224, 224)
NUM_FRAME_INPUT_LSTM = 20
NUM_CLASSIFY = 3
TRANSFER_VALUE_SIZE = 4096
RNN_SIZE = 512
EPOCH = 300
BATCH_SIZE = 150

Mình cần phải có dữ liệu để train, dữ liệu thường ở dưới máy local, phải tải lên google drive để dùng được cho colab,
dữ liệu gồm nhiều files khi tải lên google drive sẽ bất tiện. Mình cần nén nó lại thành file [*.zip] rồi tải lên 
google drive. Xong sau đó dùng code này để giải nén zip ra thì mình sẽ có dữ liệu,
Sau khi mình tải tệp zip lên google drive thì bắt đầu công việc giải nén ra một thư để thực hiện việt input train

In [ ]:
fun_extractZipFile(pathFileZip= DIR_ROOT + 'InputsZip/out3.zip', pathToSave= DIR_INPUT_TRAIN)

@ Deep Learning>  Extract All /content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/InputsZip/out3.zip
Extracting...
Extract Done


Bây giờ để đảm bảo rằng tất cả các file của chúng ta được giải nén thành công vào trong thư mục mình mong muốn. Mình cần thực hiện dòng lệnh này để xem. Wow

In [ ]:
fileNames = fun_getFileNames(path= DIR_INPUT_TRAIN)
fun_print(name= 'Xem thư mục bao nhiêu file?: ' + DIR_INPUT_TRAIN, value= 'Có: {0} Files'.format(len(fileNames)))

@ Deep Learning>  Xem thư mục bao nhiêu file?: /content/drive/My Drive/DeepLearning/Data_PhanLoaiBaoLuc/InputTrainTest/
Có: 15 Files


Wow dữ liệu của mình đã có. Bây giờ bắt đầu giai đoạn tiếp theo. Load model VGG16

Đầu là khu vực dành cho import thư viện

In [ ]:
from keras.applications import VGG16
from keras.models import Model, Sequential, load_model
import numpy as np
from random import shuffle
from keras.layers import LSTM, Dense, Activation

In [ ]:
modelCNN = VGG16(include_top= True, weights= 'imagenet')
modelCNN.summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

Mình sẽ thử dùng modelCNN để predict 20 frame ảnh xem sao.

In [ ]:
frames = fun_getFramesOfVideo(path= DIR_INPUT_TRAIN + 'fi_da_out_0.avi', count=20)

fun_print('count frame: ', value=len(frames))

transferLayer = modelCNN.get_layer(name='fc2')

imgModelTransfer = Model(inputs= modelCNN.input, outputs= transferLayer.output)

frames = fun_resizeFrames(frames= frames, size=SIZE)

frames = np.array(frames)
frames = (frames / 255.).astype(np.float16)

transfer = imgModelTransfer.predict(frames)
print(transfer)

@ Deep Learning>  count frame: 
20
[[0.         0.         1.2807381  ... 0.         0.         0.9206897 ]
 [0.         0.         1.2729297  ... 0.         0.         0.9155502 ]
 [0.         0.         1.2755383  ... 0.         0.         0.9062257 ]
 ...
 [0.         0.         1.4305899  ... 0.         0.         0.9416685 ]
 [0.         0.         1.4025955  ... 0.         0.         0.8959355 ]
 [0.         0.         1.4778105  ... 0.         0.         0.89356214]]


Bây giờ thực thực hiện định nghĩa một hàm. Nhận vào danh sách các video train. Trả về mảng tên video và mảng labels.

In [ ]:
def getVideoLabelNames(path: str):
  names = []
  labels = []
  fileNames = fun_getFileNames(path=path)
  for file in fileNames:
    names.append(file)
    if file.find('fi_danh') != -1:
      labels.append([1, 0, 0])
    elif file.find('fi_da') != -1:
      labels.append([0, 1, 0])
    elif file.find('fi_nt') != -1:
      labels.append([0, 0, 1])
  
  c = list(zip(names, labels))
  shuffle(c)

  names, labels = zip(*c)
  return names, labels

Bây giờ test thử hàm getVideoLabelNames(path: str): xem sao

In [ ]:
names, labels = getVideoLabelNames(path= DIR_INPUT_TRAIN)

fun_print('Names', names[0:3])
fun_print('labels', labels[0:3])

@ Deep Learning>  Names
('fi_danh_out_116.avi', 'fi_nt_out_214.avi', 'fi_danh_out_117.avi')
@ Deep Learning>  labels
([1, 0, 0], [0, 0, 1], [1, 0, 0])


Bây giờ mình sẽ viết một hàm nhận vào một video. Hàm này sẽ lấy 20 frame hình cho vào VGG16 để lấy mẫu dự đoán

In [ ]:
def getTransferValue(pathVideoOrListFrame):
  if isinstance(pathVideoOrListFrame, str):
    frames = fun_getFramesOfVideo(path=pathVideoOrListFrame, count=NUM_FRAME_INPUT_LSTM)
  else:
    frames = pathVideoOrListFrame
    
  frames = fun_resizeFrames(frames= frames, size=SIZE)

  frames = np.array(frames)
  frames = (frames / 255.).astype(np.float16)

  transfer = imgModelTransfer.predict(frames)
  return transfer

Cùng test thử getTransferValue(pathVideo: str): xem nào!

In [ ]:
transfer = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + 'fi_da_out_0.avi')
print(transfer)

[[0.         0.         1.2807381  ... 0.         0.         0.9206897 ]
 [0.         0.         1.2729297  ... 0.         0.         0.9155502 ]
 [0.         0.         1.2755383  ... 0.         0.         0.9062257 ]
 ...
 [0.         0.         1.4305899  ... 0.         0.         0.9416685 ]
 [0.         0.         1.4025955  ... 0.         0.         0.8959355 ]
 [0.         0.         1.4778105  ... 0.         0.         0.89356214]]


Wowww. Bây giờ mình sẽ định nghĩa một hàm nhận vào một nhãn của video (label) trả về một dạng ones hot

In [ ]:
def onesHotLabel(label: list):
  _ones = np.ones([NUM_FRAME_INPUT_LSTM, NUM_CLASSIFY])
  _onesHot = label * _ones
  return np.array(_onesHot)

Test thử hàm oneHotLabel(label: list): xem nào!



In [ ]:
_oneHot = onesHotLabel([0, 1, 0])
fun_print(name='oneHost', value= _oneHot)

@ Deep Learning>  oneHost
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


Từ nãy giờ cũng được nhiều rồi. Để đảm bảo rằng dữ liệu hợp lệ. Bạn hãy test lại một lần nửa xem đã có dữ liệu chưa. Hãy thực hiện các lệnh dưới đây.

In [ ]:
names, labels = getVideoLabelNames(path= DIR_INPUT_TRAIN)

fun_print(name= 'Size of List video', value= len(names))
fun_print(name= 'Size of List labels', value= len(labels))

print('= '*50)

print(names[0:5])
print(labels[0:5])

print('= '*50)

print(getTransferValue(DIR_INPUT_TRAIN + names[0]))
print(onesHotLabel(labels[0]))

@ Deep Learning>  Size of List video
15
@ Deep Learning>  Size of List labels
15
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
('fi_danh_out_115.avi', 'fi_nt_out_215.avi', 'fi_danh_out_116.avi', 'fi_nt_out_216.avi', 'fi_da_out_4.avi')
([1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 1], [0, 1, 0])
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
[[0.         0.         1.2836535  ... 0.         0.         0.80077934]
 [0.         0.         1.2165948  ... 0.         0.         0.91821134]
 [0.         0.         1.2474657  ... 0.         0.         0.97603726]
 ...
 [0.         0.         1.1223265  ... 0.         0.         0.7579609 ]
 [0.         0.         1.1686025  ... 0.         0.         0.70442736]
 [0.         0.         1.2218906  ... 0.         0.         0.7744684 ]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 

Nếu mọi thứ đã Ok bây giờ mình sẽ viết một hàm nhận vào số lượng video muốn train. Trả về 2 danh sách:
- TranSet: tập dữ liệu train
- LabelSet: tập dữ liệu gán nhãn cho mỗi tập trong trainSet.

Nếu mọi thứ thuận lợi thì nó là tiền đề để bắt đầu huấn luyện LSTM.

In [ ]:
def getTrainSet_LabelSet(numItem: int):
  count = 0
  trainSet = []
  labelSet = []
  while count < numItem:
    itemTrain = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + names[count])
    itemLable = onesHotLabel(label= labels[count])

    trainSet.append(itemTrain)
    labelSet.append(itemLable[0])

    fun_print_process(count= count, max= numItem)

    count += 1
  
  return trainSet, labelSet

Cùng Test Thử getTrainSet_LabelSet(numItem: int): xem nào!

In [ ]:
trainSet, labelSet = getTrainSet_LabelSet(numItem= len(names))

 - Processing:  [93.3%]

In [ ]:
print(len(trainSet))
print(len(trainSet[0]))
print(len(trainSet[0][0]))

750
20
4096


In [ ]:
print(len(labelSet))
print(labelSet[0])
print(labelSet[0][0])

750
[0. 0. 1.]
0.0


Hãy viết một hàm trả về cấu trúc model LSTM

In [ ]:
def getModelLSTM(rnn_size: int = 512, input_shape: tuple = (20, 4096), num_classify: int = 3):
  modelLSTM = Sequential()
  modelLSTM.add(LSTM(rnn_size, input_shape= input_shape))
  modelLSTM.add(Dense(1024))
  modelLSTM.add(Activation('relu'))
  modelLSTM.add(Dense(50))
  modelLSTM.add(Activation('sigmoid'))
  modelLSTM.add(Dense(num_classify))
  modelLSTM.add(Activation('softmax'))
  modelLSTM.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
  return modelLSTM

In [ ]:
model = getModelLSTM()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               9439232   
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                51250     
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 153       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0

In [ ]:
model.fit(np.array(trainSet), np.array(labelSet), epochs=EPOCH, batch_size=BATCH_SIZE, verbose=2, validation_split= 0.2)

model.save(DIR_ROOT + 'Models/LSTM_Train_Viet3.h5')

fun_print(name= 'LSTM Train', value= 'Finish')

Epoch 1/300
1/1 - 0s - loss: 0.2266 - accuracy: 0.3333 - val_loss: 0.2679 - val_accuracy: 0.0000e+00
Epoch 2/300
1/1 - 0s - loss: 0.2442 - accuracy: 0.4167 - val_loss: 0.3818 - val_accuracy: 0.0000e+00
Epoch 3/300
1/1 - 0s - loss: 0.2370 - accuracy: 0.4167 - val_loss: 0.2781 - val_accuracy: 0.3333
Epoch 4/300
1/1 - 0s - loss: 0.2321 - accuracy: 0.3333 - val_loss: 0.2306 - val_accuracy: 0.3333
Epoch 5/300
1/1 - 0s - loss: 0.2152 - accuracy: 0.3333 - val_loss: 0.2287 - val_accuracy: 0.0000e+00
Epoch 6/300
1/1 - 0s - loss: 0.2112 - accuracy: 0.5833 - val_loss: 0.2683 - val_accuracy: 0.0000e+00
Epoch 7/300
1/1 - 0s - loss: 0.2063 - accuracy: 0.4167 - val_loss: 0.2898 - val_accuracy: 0.0000e+00
Epoch 8/300
1/1 - 0s - loss: 0.1999 - accuracy: 0.4167 - val_loss: 0.2623 - val_accuracy: 0.0000e+00
Epoch 9/300
1/1 - 0s - loss: 0.1867 - accuracy: 0.5000 - val_loss: 0.2418 - val_accuracy: 0.3333
Epoch 10/300
1/1 - 0s - loss: 0.1788 - accuracy: 0.6667 - val_loss: 0.2596 - val_accuracy: 0.3333
Epoch

Wow. Bây giờ hãy cùng test xem độ chính xác của model nào.

In [ ]:
namesVald, labelsVald = getVideoLabelNames(path= DIR_ROOT + 'InputTrainTest/')

print(len(namesVald))

15


In [ ]:
def getValdSet_ValdLabelSet(numItem: int):
  count = 0
  valdSet = []
  valdLabel = []
  while count < numItem:
    itemVald = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + namesVald[count])
    itemLabel = onesHotLabel(label= labelsVald[count])

    valdSet.append(itemVald)
    valdLabel.append(itemLabel[0])

    count += 1

  return valdSet, valdLabel

In [ ]:
valdSet, valdLabelSet = getValdSet_ValdLabelSet(numItem= len(namesVald))

In [ ]:
result = model.evaluate(np.array(valdSet), np.array(valdLabelSet))

1/1 [==============================] - 0s 2ms/step - loss: 0.1753 - accuracy: 0.7333


In [ ]:
for name, value in zip(model.metrics_names, result):
  print(name, value)

loss 0.17530281841754913
accuracy 0.7333333492279053


Woww. Hãy cùng thử predict với từng video và xem kết quả nào

In [ ]:
for file in namesVald:
  frames = getTransferValue(pathVideoOrListFrame= DIR_INPUT_TRAIN + file)
  pre = model.predict(np.array([frames]))
  fun_rint(name= 'predict: ' + file + ' | {0}'.format(np.argmax(pre)), value= pre)
  print('\r')

@ Deep Learning>  predict: fi_nt_out_215.avi | 2
[[0.0367764  0.05087447 0.91234916]]

@ Deep Learning>  predict: fi_danh_out_117.avi | 0
[[0.9572867  0.00143663 0.04127667]]

@ Deep Learning>  predict: fi_danh_out_116.avi | 0
[[0.9916529  0.00099888 0.00734821]]

@ Deep Learning>  predict: fi_da_out_3.avi | 1
[[0.00610643 0.87416935 0.11972412]]

@ Deep Learning>  predict: fi_nt_out_216.avi | 2
[[0.00543498 0.00628432 0.9882808 ]]

@ Deep Learning>  predict: fi_danh_out_119.avi | 0
[[0.9780797  0.0012411  0.02067921]]

@ Deep Learning>  predict: fi_nt_out_213.avi | 2
[[0.00474691 0.03336468 0.96188843]]

@ Deep Learning>  predict: fi_danh_out_115.avi | 0
[[0.99157286 0.00244107 0.0059861 ]]

@ Deep Learning>  predict: fi_da_out_2.avi | 2
[[0.01611722 0.05550494 0.92837787]]

@ Deep Learning>  predict: fi_nt_out_214.avi | 2
[[0.00409361 0.01832251 0.9775839 ]]

@ Deep Learning>  predict: fi_danh_out_118.avi | 2
[[0.09483974 0.00731582 0.89784443]]

@ Deep Learning>  predict: fi_da_out_

Bây giờ hãy thử put text và save video về thử xem.

In [ ]:
def fun_predict(model, tranfer_value):
    arrPre = []
    arrPre.append(tranfer_value)
    Real = model.predict(np.array(arrPre))
    pre = np.argmax(Real)

    print(Real, pre)
    print('\r')
    return pre, Real[0][pre]

#version 1
def fun_loadVideoPredictPutTextAndSave(modelCNN, modelLSTM, pathLoadVideo, pathSave, perFameInputLSTM= 20, tranferSize: int = 4096):
    if pathLoadVideo is None or pathSave is None or perFameInputLSTM is None or modelCNN is None:
        return

    cap = cv2.VideoCapture(pathLoadVideo)
    isContinue, frame = cap.read()

    size = fun_getSizeOfFrame(frame= frame)
    wr = cv2.VideoWriter(pathSave, cv2.VideoWriter_fourcc(*'MJPG'), 25, size)
    countFrame = 0

    while True:
      if not isContinue:
        break
      
      imgs = []
      trans = []
      count = perFameInputLSTM

      while count > 0:
        imgs.append(frame)
        trans.append(frame)
        isContinue, frame = cap.read()
        count -= 1

      trans = getTransferValue(trans)

      pre, real = fun_predict(model= model, tranfer_value= trans)

      color=(100, 200, 255)
      conv = ''
      if pre == 0:
        conv = 'Danh'
        color = (150, 250, 255)
      elif pre == 1:
        color = (200, 100, 255)
        conv = 'Da'
      elif pre == 2:
        conv = 'Nam Toc'

      text = 'Predict: {0} -> Real: [ {1} ]'.format(conv, real)

      for ff in imgs:
        countFrame += 1
        # putText
        cv2.putText(img=ff,
                    text=text,
                    org=(50, 100),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    thickness=2,
                    color= color )

        cv2.putText(img=ff,
                    text='Frame Count: {0}'.format(countFrame),
                    org=(50, 150),
                    fontScale=1,
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    thickness=2,
                    color= color )

        wr.write(ff)

      isContinue, frame = cap.read()

    wr.release()
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
model = load_model(DIR_MODEL_LSTM)
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 512)               9439232   
_________________________________________________________________
dense_57 (Dense)             (None, 1024)              525312    
_________________________________________________________________
activation_57 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_58 (Dense)             (None, 50)                51250     
_________________________________________________________________
activation_58 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 3)                 153       
_________________________________________________________________
activation_59 (Activation)   (None, 3)               

In [ ]:
fun_loadVideoPredictPutTextAndSave(modelCNN= imgModelTransfer, modelLSTM= model, pathLoadVideo= DIR_ROOT + 'Tests/video_test8.mp4',
                                   pathSave= DIR_ROOT + 'Tests/video_test8_out.avi', perFameInputLSTM= 20)

[[9.9679810e-01 2.3827578e-03 8.1917184e-04]] 0

[[0.9936528  0.00311101 0.00323608]] 0

[[0.9975943  0.00136221 0.00104338]] 0

[[0.9292684  0.06835053 0.00238107]] 0

[[0.95527315 0.04316957 0.00155733]] 0

[[0.9737781  0.02084633 0.00537547]] 0

[[0.3366834  0.00738402 0.6559326 ]] 2

[[0.99066013 0.00832519 0.00101474]] 0

[[0.99744886 0.0015468  0.00100428]] 0

!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
!Error To Resize of 20
[[0.99735546 0.00117169 0.00147288]] 0

